# NVIDIA FLARE Hello World Examples

Welcome to NVIDIA FLARE Hello World examples! This notebook demonstrates how to use **Job Recipes** to quickly build and run federated learning applications.

It is recommended to create a virtual environment and run everything within a virtualenv.

## Table of Contents

1. [Prerequisites](#Prerequisites)
2. [Example 1: Hello PyTorch](#Example-1:-Hello-PyTorch) - Image classification with CIFAR-10
3. [Example 2: Hello NumPy](#Example-2:-Hello-NumPy) - Understanding FL basics
4. [Example 3: Hello PyTorch Lightning](#Example-3:-Hello-PyTorch-Lightning) - Lightning integration
5. [Example 4: Hello TensorFlow](#Example-4:-Hello-TensorFlow) - MNIST digit classification
6. [Example 5: Hello Flower](#Example-5:-Hello-Flower) - Flower on FLARE infrastructure
7. [Summary](#Summary)

## What's New in FLARE 2.7

NVIDIA FLARE now features **Job Recipes** - a simplified API that makes federated learning as easy as running a Python script. Instead of manually configuring workflows and components, you simply:

1. Define your model and training script
2. Create a Job Recipe (5-10 lines of code)
3. Run it: `python job.py`

## Quick Start

Each example can be run directly from its directory:

```bash
cd hello-pt
pip install -r requirements.txt
python job.py
```

The job will run in the simulator with multiple clients, and results will be saved to `/tmp/nvflare/simulation/[job-name]/`.

## What is a Job Recipe?

A Job Recipe is a high-level API introduced in FLARE v2.7 that encapsulates the entire FL workflow:

```python
from nvflare.app_opt.pt.recipes.fedavg import FedAvgRecipe
from nvflare.recipe import SimEnv

recipe = FedAvgRecipe(
    name="hello-pt",
    min_clients=2,
    num_rounds=2,
    initial_model=SimpleNetwork(),
    train_script="client.py",
)

env = SimEnv(num_clients=2)
run = recipe.execute(env)
```

That's it! The same recipe works in simulation, POC, and production environments.

## Next Steps

After trying these examples:
- 📚 Explore [step-by-step examples](./step-by-step/) for specific FL techniques
- 🔄 Learn [ML-to-FL conversion](./ml-to-fl/) to adapt your existing code
- 📖 Read the [Job Recipe documentation](https://nvflare.readthedocs.io/en/main/user_guide/data_scientist_guide/job_recipe.html) for advanced usage
- 🚀 Check out [advanced examples](../advanced/) for production scenarios

### More Hello World Examples

Beyond the 5 examples above, explore additional hello-world examples:
- 🔄 [Hello Cyclic Weight Transfer](./hello-cyclic/) - Cyclic learning workflow
- ✅ [Hello Cross-Validation](./hello-numpy-cross-val/) - Cross-site validation
- 🎯 [Hello Client Controlled Workflow](./hello-ccwf/) - Client-driven FL
- 📊 [Hello Tabular Statistics](./hello-tabular-stats/) - Federated statistics
- 🔐 [Hello Learning Rate](./hello-lr/) - Adaptive learning rates
- 🖼️ [Hello ResNet](./hello-pt-resnet/) - ResNet with PyTorch

## Prerequisites

To run the examples:

1. **Install NVIDIA FLARE:**
   ```bash
   pip install nvflare
   ```
   
   **Optional - Install with framework support:**
   ```bash
   pip install nvflare[PT]  # For PyTorch examples
   pip install nvflare[TF]  # For TensorFlow examples
   ```

2. **Get the example code:**
   ```bash
   git clone https://github.com/NVIDIA/NVFlare.git
   git switch <release branch>
   cd examples/hello-world
   ```
  If running in google colab, running the following to get the source code
  
  %pip install --ignore-installed blinker
  ! npx degit -f NVIDIA/NVFlare/examples/hello-world . -y

3. **Install example dependencies:**
   
   Each example has its own `requirements.txt`. Navigate to the example directory and install:
   ```bash
   cd hello-pt  # or hello-numpy, hello-lightning, etc.
   pip install -r requirements.txt
   ```

4. **Run the example:**
   ```bash
   python job.py
   ```

That's it! The simulator will run locally with multiple clients, and results will be saved to `/tmp/nvflare/simulation/[job-name]/`.

---

# Examples

The following sections demonstrate federated learning with different frameworks using NVIDIA FLARE Job Recipes. Each example can be run directly from the terminal with `python job.py`.


## Example 1: Hello PyTorch

📁 **[View full example code and README](./hello-pt/)**

This example trains an image classifier on [CIFAR-10](https://www.cs.toronto.edu/~kriz/cifar.html) using PyTorch and federated averaging ([FedAvg](https://arxiv.org/abs/1602.05629)).

### What You'll Learn
- Using the `FedAvgRecipe` for PyTorch
- Running with the Simulator
- Client API integration
- TensorBoard metric tracking

### Dataset

This example uses [CIFAR-10](https://www.cs.toronto.edu/~kriz/cifar.html), a dataset of 60,000 32x32 color images in 10 classes. The dataset is automatically downloaded via torchvision's datasets module.

In a real FL experiment, each client would have their own dataset. Here for simplicity, we use the same dataset on each client.

### Code Structure

```
hello-pt/
├── client.py         # Client training code
├── model.py          # Model definition
├── job.py            # Job recipe
└── requirements.txt  # Dependencies
```


### Model Architecture

The example uses a simple CNN for CIFAR-10 classification. In PyTorch, neural networks are implemented by defining a class that extends `nn.Module`. The network's architecture is set up in the `__init__` method, while the `forward` method determines how input data flows through the layers:


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SimpleNetwork(nn.Module):
    def __init__(self):
        super(SimpleNetwork, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

print("✅ Model defined")


### Client Code

The client code (`client.py`) uses NVIDIA FLARE's Client API to integrate with federated learning. The training code is almost identical to standard PyTorch training, with just a few lines added to communicate with the server.

**Client API Pattern - Three Essential Methods:**
- `flare.init()` - Initialize the FLARE Client API environment
- `flare.receive()` - Receive the global model from the FL server
- `flare.send()` - Send the updated model back to the FL server

With these simple methods, you can convert centralized training code to federated learning with minimal changes.

### Job Recipe

The job recipe defines the complete FL workflow in just a few lines:


In [ ]:
# This shows the recipe pattern - actual execution would be in hello-pt/job.py
from nvflare.app_opt.pt.recipes.fedavg import FedAvgRecipe
from nvflare.recipe import SimEnv, add_experiment_tracking

# Create the recipe
recipe = FedAvgRecipe(
    name="hello-pt",
    min_clients=2,
    num_rounds=2,
    initial_model=SimpleNetwork(),
    train_script="client.py",
)

# Add TensorBoard tracking
add_experiment_tracking(recipe, tracking_type="tensorboard")

print("✅ Recipe created")
print(f"   Job name: {recipe.name}")
print(f"   Clients: 2")
print(f"   Rounds: 2")


### Running the Example

To run this example from the terminal:

```bash
cd hello-pt
pip install -r requirements.txt
python job.py
```

The job will:
1. Start a simulation environment with 2 clients
2. Distribute the initial model to clients
3. Run 2 rounds of federated training
4. Save results to `/tmp/nvflare/simulation/hello-pt/`

**View TensorBoard metrics:**
```bash
tensorboard --logdir /tmp/nvflare/simulation/hello-pt/server/simulate_job/tb_events
```

### Expected Output

After running, you should see:
- **Round 0 and Round 1**: Training logs with loss reported for each epoch
- **Model aggregation**: Server aggregates models from both clients
- **Final model**: Saved to `/tmp/nvflare/simulation/hello-pt/`
- **TensorBoard logs**: Available at `/tmp/nvflare/simulation/hello-pt/server/simulate_job/tb_events`


## Example 2: Hello NumPy

📁 **[View full example code and README](./hello-numpy/)**

This example demonstrates federated learning with a simple NumPy model, perfect for understanding FL concepts without deep learning complexity.

### What You'll Learn
- Using the `NumpyFedAvgRecipe`
- Understanding FedAvg aggregation
- Simple FL workflow visualization

### Dataset

This example uses a simplified synthetic dataset. Each client starts with the same 3x3 weight matrix:

```
[[1., 2., 3.],
 [4., 5., 6.],
 [7., 8., 9.]]
```

During training, each client adds 1.0 to each weight. After aggregation, you'll see the weights increase by approximately 1.0 per round, clearly demonstrating the federated averaging process.

### How It Works

The `SimpleNumpyModel` class implements a basic model using NumPy arrays. Each client performs basic operations on the weight matrix, and the server averages the results.


In [ ]:
import numpy as np

class SimpleNumpyModel:
    def __init__(self):
        self.weights = np.array([[1., 2., 3.],
                                 [4., 5., 6.],
                                 [7., 8., 9.]], dtype=np.float32)
    
    def train_step(self, learning_rate=1.0):
        # Simulate training by adding learning_rate
        self.weights += learning_rate
        return self.weights

model = SimpleNumpyModel()
print("Initial weights:")
print(model.weights)
print("\nAfter training step:")
print(model.train_step(learning_rate=1.0))


### Running the Example

```bash
cd hello-numpy
pip install -r requirements.txt
python job.py
```

After 3 rounds, weights increase by approximately 3 (depending on aggregation). Results saved to `/tmp/nvflare/simulation/hello-numpy/`.


## Example 3: Hello PyTorch Lightning

📁 **[View full example code and README](./hello-lightning/)**

This example shows PyTorch Lightning integration with minimal code changes, training on [CIFAR-10](https://www.cs.toronto.edu/~kriz/cifar.html).

### What You'll Learn
- Using `LightningModule` with FLARE
- The `flare.patch()` method
- Automatic training/validation/testing

### Key Concepts

**LightningDataModule**: Encapsulates data loading, transforming, and splitting. It separates data-related logic from model code, promoting better code organization and reusability.

**LightningModule**: A high-level abstraction built on PyTorch that streamlines model training. It encapsulates the model architecture, training logic, validation, testing, and optimizer configuration.

### Data Preparation

Before running, pre-download the CIFAR-10 dataset to prevent race conditions when multiple clients try to download simultaneously:

```bash
./prepare_data.sh
```

### Key Integration

PyTorch Lightning integration requires just one line - `flare.patch(trainer)`:


In [ ]:
# Lightning client integration pattern
lightning_pattern = '''
import nvflare.client.lightning as flare
from pytorch_lightning import Trainer

# Create trainer
trainer = Trainer(max_epochs=2)

# Patch trainer for FL - this is the key line!
flare.patch(trainer)

# Standard Lightning training loop
while flare.is_running():
    input_model = flare.receive()
    trainer.validate(model, datamodule)
    trainer.fit(model, datamodule)
    trainer.test(model, datamodule)
'''

print("PyTorch Lightning Integration:")
print(lightning_pattern)


### Running the Example

**Note:** Pre-download CIFAR-10 data first:

```bash
cd hello-lightning
./prepare_data.sh
pip install -r requirements.txt
python job.py --num_rounds 2 --batch_size 16
```


## Example 4: Hello TensorFlow

📁 **[View full example code and README](./hello-tf/)**

This example demonstrates TensorFlow integration using the [MNIST](https://www.tensorflow.org/datasets/catalog/mnist) handwritten digits dataset.

### What You'll Learn
- Using the `TFFedAvgRecipe`
- GPU memory management for multi-client scenarios
- TensorFlow-specific considerations

### Model Architecture

The model uses TensorFlow's Keras API with a simple sequential architecture:
- **Flatten Layer**: Prepares 28x28 images for dense layers
- **Dense Layer**: 128 units with ReLU activation for non-linearity
- **Dropout Layer**: 20% dropout rate to prevent overfitting
- **Output Layer**: 10 units for digit classification (0-9)

### GPU Memory Management

TensorFlow attempts to allocate all available GPU memory at startup. For multi-client simulation, set these flags to allow dynamic memory growth:

```bash
TF_FORCE_GPU_ALLOW_GROWTH=true TF_GPU_ALLOCATOR=cuda_malloc_async
```

**Optional:** If you have more GPUs than clients, run one client per GPU using:
```bash
nvflare simulator -n 2 --gpu 0,1 [job]
```

### Running the Example

```bash
cd hello-tf
pip install -r requirements.txt
TF_FORCE_GPU_ALLOW_GROWTH=true TF_GPU_ALLOCATOR=cuda_malloc_async python job.py
```

Results saved to `/tmp/nvflare/simulation/hello-tf/`.


## Example 5: Hello Flower

📁 **[View full example code and README](./hello-flower/)**

This example shows how to run [Flower](https://flower.ai/) applications on NVIDIA FLARE infrastructure, combining Flower's simplicity with FLARE's enterprise features.

### What You'll Learn
- Integrating Flower with NVIDIA FLARE
- Using Flower's `ClientApp` and `ServerApp`
- Optional TensorBoard metric streaming

### Dataset

This example uses [CIFAR-10](https://www.cs.toronto.edu/~kriz/cifar.html) with a simple CNN based on PyTorch's "60 Minute Blitz" tutorial.

### Code Structure

```
hello-flower/
├── flwr-pt/              # Basic Flower PyTorch app
│   └── flwr_pt/
│       ├── client.py     # Contains ClientApp
│       ├── server.py     # Contains ServerApp
│       └── task.py       # Model and data definitions
├── flwr-pt-tb/           # With TensorBoard streaming
│   └── flwr_pt_tb/
│       ├── client.py     # ClientApp with metrics
│       ├── server.py     # ServerApp
│       └── task.py       # Model and data definitions
└── job.py                # FLARE Job Recipe
```

The client code in `client.py` contains the **Flower Client App**, while `server.py` uses Flower's built-in FedAvg **Strategy**.

### Running the Example

**Basic Flower app:**
```bash
cd hello-flower
pip install -r requirements.txt
python job.py --job_name "flwr-pt" --content_dir "./flwr-pt"
```

**With TensorBoard streaming:**
```bash
python job.py --job_name "flwr-pt-tb" --content_dir "./flwr-pt-tb" --stream_metrics
tensorboard --logdir /tmp/nvflare/hello-flower-tb
```


---

## Summary

You've learned how to use NVIDIA FLARE Job Recipes to build federated learning applications across different frameworks:

✅ **PyTorch** - Image classification with FedAvg  
✅ **NumPy** - Understanding FL basics  
✅ **PyTorch Lightning** - Minimal integration with `flare.patch()`  
✅ **TensorFlow** - MNIST with GPU memory management  
✅ **Flower** - Running Flower apps on FLARE infrastructure  

### Key Takeaways

1. **Job Recipes are simple** - Define FL jobs in 5-10 lines of code
2. **Same code, multiple environments** - Recipes work in simulation, POC, and production
3. **Framework flexibility** - Use PyTorch, TensorFlow, NumPy, Lightning, or Flower
4. **Easy to run** - Just `python job.py` in any example directory

### Learn More

- **Documentation**: [https://nvflare.readthedocs.io/](https://nvflare.readthedocs.io/)
- **GitHub**: [https://github.com/NVIDIA/NVFlare](https://github.com/NVIDIA/NVFlare)
- **Example Catalog**: [https://nvidia.github.io/NVFlare/catalog/](https://nvidia.github.io/NVFlare/catalog/)

### Get Help

- **GitHub Issues**: [https://github.com/NVIDIA/NVFlare/issues](https://github.com/NVIDIA/NVFlare/issues)
- **Discussions**: [https://github.com/NVIDIA/NVFlare/discussions](https://github.com/NVIDIA/NVFlare/discussions)

**Happy Federated Learning! 🚀**
